Project script is made for reading several .txt files, containing some data in several columns, converting them into valid dataframe and sorting values according to the relevant column.
Files need to be processed and combined into one .csv file, data needs to be wrangled and excluded of Nan values.

In [1]:
import os
import pandas as pd
import re

Initializing empty Dataframe and function which will add empty row at the end of each .txt file, since last row is always shown as bad one and Pandas skips it.

In [2]:
combined_files = pd.DataFrame()
def add_rows_to_text_file(file_path):
    with open(file_path, 'a') as file:
        file.write('\n')

In [3]:
directory = "C:/Users\Anja/Downloads/relative importance inputs"

Function which needs to split each row by two conditions: 1. Whitespaces between each data columns need to be >2; 2. Int or float values are read as one without splitting. However, there are several rows that had only 1 whitespace between them, which caused me to build another split_col function to further split rows.

In [4]:
def split_string(x):
    if isinstance(x, str):
        if re.search(r'\s{2,}', x):
            return re.split(r'\s{2,}', x)
    return x

def split_col(x):
    return x.split(maxsplit=1)


Processing/ reading files into Dataframe, skipping header rows (7) and creating rows with tab as delimiter. All of files are then concating into one Dataframe.

In [ ]:
def process_and_combine_files(directory):
    combined_data = pd.DataFrame()  # Initialize an empty DataFrame to store the combined data

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            #add_rows_to_text_file(file_path)
            year = filename.split('.')[0][-4:]
            data = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip', skiprows=7)
            data.columns = ['Podaci']
            data['Podaci'] = data['Podaci'].apply(split_string)
            data = data['Podaci'].apply(pd.Series)
            del data[data.columns[-1]]
            data.columns = ['Item Code', 'Item and Group', 'CPI-U', 'CPI-W']
            data['Year'] = year
            combined_data = pd.concat([combined_data, data], ignore_index=True)

    return combined_data

I've used this piece of code to try my function on single .txt file, so I can test and correct errors in code and final data.

In [15]:
file_path = os.path.join(directory, "1988.txt")
out_dir_proba = "E:/"
data1 = pd.read_csv(file_path, delimiter="\t", on_bad_lines='skip', skiprows=7)
data1.columns = ['Podaci']
data1['Podaci'] = data1['Podaci'].apply(split_string)
data1 = data1['Podaci'].apply(pd.Series)
out_proba = os.path.join(out_dir_proba, 'one_file.csv')
data1.to_csv(out_proba, index=False)

Combining files created several rows that had Nan values ( usually data that was represented  in 2 rows). Correcting rows is merging 2 rows into one, combining first column (string) into one string and merging float values into one value. Then, I erased rows which had all Nan values in second and third column.

In [25]:
df = process_and_combine_files(directory)
bad_rows = df[df['CPI-W']== '']
def correct_rows(rows):
    for i in range(len(rows.index)):
        change = split_col(str(rows.iloc[i][0]))
        rows.iloc[i]= [change[0], change[1], rows.iloc[i][1], rows.iloc[i][2], rows.iloc[i][4]]
    return rows
corrected = correct_rows(bad_rows)
df.update(corrected)
df.head()

,Item Code,Item and Group,CPI-U,CPI-W,Year
0,SAC,COMMODITIES,45.531,49.440,1987
1,SACE,ENERGY COMMODITIES,3.716,4.418,1987
2,SACL1,COMMODITIES LESS FOOD,29.476,31.648,1987
3,SACL1E,COMMODITIES LESS FOOD AND ENERGY,25.760,27.230,1987
4,SACL1E4,"COMMODITIES LESS FOOD, ENERGY, & UCR",24.447,24.884,1987


Deduping files by each column to see if there are any duplicates.

In [31]:
def dedupe(files):
    duplicates_by_year = files.duplicated(subset=['Year', 'Item Code', 'Item and Group', 'CPI-U', 'CPI-W'], keep=False)
    return duplicates_by_year

deduped = dedupe(df)

# Check if there are any True values in deduped Series
if deduped.any():
    count_of_duplicates = deduped.sum()
    print("There are {} duplicate rows within each year.".format(count_of_duplicates))
else:
    print("There are no duplicate rows within each year.")


There are no duplicate rows within each year.


Quick piece of code if there were to happen that we had duplicates and wanted to save original and deduped data into 2 separate sheets.

out_dir = "E:/"

output_csv_file = os.path.join(out_dir, "combined_files.xlsx")

with pd.ExcelWriter(output_csv_file) as writer:
    # Save the original DataFrame to the first sheet (Sheet Name: 'Original Data')
    df.to_excel(writer, sheet_name='Combined Files', index=False)
    deduped.to_excel(writer, sheet_name='Duplicated', index=False)

In [33]:
df.sort_values(by='Item and Group', ascending=False)

,Item Code,Item and Group,CPI-U,CPI-W,Year
3359,SE3805,WOMEN'S SUITS,0.165,0.108,1995
2969,SE3805,WOMEN'S SUITS,0.179,0.117,1994
1799,SE3805,WOMEN'S SUITS,0.184,0.121,1991
629,SE3805,WOMEN'S SUITS,0.175,0.118,1988
1409,SE3805,WOMEN'S SUITS,0.172,0.115,1990
...,...,...,...,...,...
2699,SE6203,ADMISSIONS,0.683,0.586,1993
1919,SE6203,ADMISSIONS,0.683,0.584,1991
2309,SE6203,ADMISSIONS,0.689,0.589,1992
3879,SE6203,ADMISSIONS,0.714,0.613,1996


In [32]:
out_dir = "E:/"

output_csv_file = os.path.join(out_dir, "combined_files.csv")

df.to_csv(output_csv_file, index=False)